In [1]:
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer

# Load WikiText-103 dataset
dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")

Found cached dataset wikitext (/home/songx_lab/cse12012530/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


In [2]:
dataset.save_to_disk("wikitext-103-raw-train")

Saving the dataset (0/2 shards):   0%|          | 0/1801350 [00:00<?, ? examples/s]

In [2]:


# tokenizer_name = "gpt2"
# # Initialize tokenizer
# tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2") # 论文有说是increase 到1024


In [2]:
print(tokenizer.vocab_size)

50257


In [29]:
print("Tokenizer max length: ", tokenizer.model_max_length) 
tokenizer.model_max_length = 128
print("Tokenizer max length after change: ", tokenizer.model_max_length) 

Tokenizer max length:  128
Tokenizer max length after change:  128


In [5]:
# from transformers import GPT2Tokenizer, GPT2Config
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [6]:
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 1801350
})


In [7]:
for i in range(5):
    print(dataset[i])  #  这里返回的是一个字典，包含了text，而且这里只有text

{'text': ''}
{'text': ' = Valkyria Chronicles III = \n'}
{'text': ''}
{'text': ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n'}
{'text': " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adju

In [ ]:
# for i in range(5):
#     sentence = dataset[i]
#     # print(tokenizer.tokenize(sentence['text']))  # tokenize 就是分词
#     # print(tokenizer.__call__(sentence['text'], padding="max_length", truncation=True))  # padding 这里就是补充很多到最大长度
#     if len(sentence['text']) < 5:
#         print(sentence['text'],"too short ")
#         continue
#     temp = tokenizer(sentence['text'], padding="max_length", truncation=True)   # 也就是截断和扩充是在这里完成的,这里的长度是512，但是其实你在实际的模型还可以继续截断
#     print(len(temp['input_ids']))
 

In [9]:
def filter_function(example): 
    return len(example['text'].split()) >= 5

dataset_without_short = dataset.filter(filter_function)


Loading cached processed dataset at /home/songx_lab/cse12012530/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-052d5231da8dcc7f.arrow


In [11]:
print(len(dataset_without_short))

1127372


In [ ]:
# print(tokenizer.vocab) # 这个真的就是有很多

In [30]:
if tokenizer.pad_token is None:
    # tokenizer.set_special_tokens({'pad_token': '[PAD]'})
    tokenizer.pad_token = tokenizer.eos_token

Using pad_token, but it is not set yet.


In [39]:

# Define function to tokenize and encode text
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=tokenizer.model_max_length)
    

# Apply preprocessing to dataset
preprocessed_dataset_without_short = dataset_without_short.map(preprocess_function, batched=True)

# Split preprocessed dataset into train, validation, and test sets
splits = preprocessed_dataset_without_short.train_test_split(test_size=0.1, shuffle=True)
preprocessed_splits_without_short = DatasetDict({
    "train": splits["train"],
    "validation": splits["test"],
    "test": load_dataset("wikitext", "wikitext-103-raw-v1", split="test")
})



Map:   0%|          | 0/1127372 [00:00<?, ? examples/s]

In [ ]:
preprocessed_splits_without_short.save_to_disk(f"wikitext-103-preprocessed-ws-{tokenizer.name_or_path}-{tokenizer.model_max_length}")


Flattening the indices:   0%|          | 0/1014634 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/1014634 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/112738 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/112738 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4358 [00:00<?, ? examples/s]

In [37]:
# Save preprocessed dataset
# preprocessed_splits_without_short.save_to_disk(f"wikitext-103-preprocessed{tokenizer.name_or_path}")
def preprocess_function(examples):
    return tokenizer.__call__(examples["text"], padding="max_length", truncation=True, max_length=tokenizer.model_max_length)
preprocessed_dataset_without_short_notext = preprocessed_dataset_without_short.remove_columns("text")
splits = preprocessed_dataset_without_short_notext.train_test_split(test_size=0.1, shuffle=True)
preprocessed_splits_without_short_notext = DatasetDict({
    "train": splits["train"],
    "validation": splits["test"],
    "test": load_dataset("wikitext", "wikitext-103-raw-v1", split="test")
})
preprocessed_splits_without_short_notext.save_to_disk(f"wikitext-103-preprocessed-ws-notext-{tokenizer.name_or_path}-{tokenizer.model_max_length}")

Found cached dataset wikitext (/home/songx_lab/cse12012530/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


Flattening the indices:   0%|          | 0/1014634 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/1014634 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/112738 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/112738 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4358 [00:00<?, ? examples/s]

In [38]:
print(len(preprocessed_dataset_without_short_notext[0]["input_ids"]))

128


In [9]:

# Define function to tokenize and encode text
def preprocess_function(examples):

    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply preprocessing to dataset
preprocessed_dataset = dataset.map(preprocess_function, batched=True)

# Split preprocessed dataset into train, validation, and test sets
splits = preprocessed_dataset.train_test_split(test_size=0.1, shuffle=True)
preprocessed_splits = DatasetDict({
    "train": splits["train"],
    "validation": splits["test"],
    "test": load_dataset("wikitext", "wikitext-103-raw-v1", split="test")
})
print(len(preprocessed_dataset)) # 1801350 条

Map:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Found cached dataset wikitext (/home/songx_lab/cse12012530/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


1801350


In [ ]:
print(preprocessed_dataset[0])

In [ ]:
# Save preprocessed dataset
preprocessed_splits.save_to_disk("wikitext-103-preprocessed")

In [2]:
preprocessed_splits = load_from_disk("wikitext-103-preprocessed")

In [3]:
print(preprocessed_splits)

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1621215
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 180135
    })
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
})


In [4]:
print(preprocessed_splits["train"][0])  # 到这一步都是完全没有mask的，只是把text转换成了id

{'text': ' Samuel Burnside was a prominent lawyer in Worcester who studied law at Dartmouth College in the early 1800s . Burnside had three daughters , Sophia , Harriet , and Elizabeth , who went on to be called by Frederick Clifton Pierce " the most notable figures in the life of Worcester . " The notability of the three daughters was due in part to the prestige and wealth Samuel Burnside had accrued as Chief Justice of the Court of Common Pleas . Both before and after Samuel Burnside \'s death on July 25 , 1850 , his three daughters were known for charitable works , having donated public gardens and a library to the city of Worcester . One daughter \'s most well @-@ known donation to the city came in the form of a bequest in her will . In 1904 , Harriet Burnside died and left $ 5 @,@ 000 in her will to the city of Worcester to build a fountain as a memorial to her father . \n', 'input_ids': [101, 5212, 7641, 5178, 2001, 1037, 4069, 5160, 1999, 12539, 2040, 3273, 2375, 2012, 16960, 22

In [15]:
print(tokenizer)

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [5]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig, GPT2Tokenizer
context_length = 128
# Initialize tokenizer
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
print(tokenizer.eos_token_id)
config = AutoConfig.from_pretrained(  
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

50256


In [9]:
import torch
text = " Samuel Burnside was a prominent lawyer in a a a  "
encoded = tokenizer.encode(text, add_special_tokens=True)
input_ids = torch.tensor(encoded).unsqueeze(0) 



# Print results
print(f"Input: {input_ids}")


Input: tensor([[17100,  8942,  1589,   373,   257,  9208,  6853,   287,   257,   257,
           257,   220,   220]])


In [5]:
print(tokenizer)

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [8]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 109.3M parameters


In [9]:
# Import the necessary libraries
from transformers import DataCollatorForLanguageModeling

# Set the padding token to the EOS token
tokenizer.pad_token = tokenizer.eos_token

# Create the data collator
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [10]:
print(data_collator) # 你可已看出
#DataCollatorForLanguageModeling(tokenizer=GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024,
#  is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>",
#  rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, 
# lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False,
#  single_word=False, normalized=True), 'pad_token': '<|endoftext|>'}), mlm=False, 
# mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

DataCollatorForLanguageModeling(tokenizer=GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|endoftext|>'}), mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')


In [12]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="codeparrot-ds",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=preprocessed_splits["train"],
    eval_dataset=preprocessed_splits["validation"],
)

In [13]:
trainer.train()

/home/songx_lab/cse12012530/transformers/src/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [ ]:
# 就是不要被拆分
keytoken_ids = []
for keyword in [
    "plt",
    "pd",
    "sk",
    "fit",
    "predict",
    " plt",
    " pd",
    " sk",
    " fit",
    " predict",
    "testtest",
]:
    ids = tokenizer([keyword]).input_ids[0]
    if len(ids) == 1:
        keytoken_ids.append(ids[0])
    else:
        print(f"Keyword has not single token: {keyword}")

In [ ]:
from torch.nn import CrossEntropyLoss
import torch

# 对于某个token，如果它是关键词，那么它的loss就是它的loss乘以一个权重
def keytoken_weighted_loss(inputs, logits, keytoken_ids, alpha=1.0):
    # Shift so that tokens < n predict n
    shift_labels = inputs[..., 1:].contiguous()
    shift_logits = logits[..., :-1, :].contiguous()
    # Calculate per-token loss
    loss_fct = CrossEntropyLoss(reduce=False)
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    # Resize and average loss per sample
    loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)
    # Calculate and scale weighting
    weights = torch.stack([(inputs == kt).float() for kt in keytoken_ids]).sum(
        axis=[0, 2]
    )
    weights = alpha * (1.0 + weights)
    # Calculate weighted average
    weighted_loss = (loss_per_sample * weights).mean()
    return weighted_loss

In [ ]:
from tqdm.notebook import tqdm

gradient_accumulation_steps = 8
eval_steps = 5_000

model.train()
completed_steps = 0
for epoch in range(num_train_epochs):
    for step, batch in tqdm(
        enumerate(train_dataloader, start=1), total=num_training_steps
    ):
        logits = model(batch["input_ids"]).logits
        loss = keytoken_weighted_loss(batch["input_ids"], logits, keytoken_ids)
        if step % 100 == 0:
            accelerator.print(
                {
                    "lr": get_lr(),
                    "samples": step * samples_per_step,
                    "steps": completed_steps,
                    "loss/train": loss.item() * gradient_accumulation_steps,
                }
            )
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        if step % gradient_accumulation_steps == 0:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1
        if (step % (eval_steps * gradient_accumulation_steps)) == 0:
            eval_loss, perplexity = evaluate()
            accelerator.print({"loss/eval": eval_loss, "perplexity": perplexity})
            model.train()
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
            if accelerator.is_main_process:
                tokenizer.save_pretrained(output_dir)
                repo.push_to_hub(
                    commit_message=f"Training in progress step {step}", blocking=False
                )